In [81]:
# Variables for configuration
generate_pandas_profiling_reports = True 

In [82]:
import pandas as pd
from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


# set pandas to show all columns of the df when using the display function
pd.set_option('display.max_columns', None)

In [83]:
# list the datasets that should be used in the current run
datasets = ["hungarian", "cleveland", "switzerland", "long-beach-va"]

In [84]:
# custom function to read the dataset into a csv formated string
# the name is used as a delimiter here because it is the last feature and common among all entrys 
def readRawData(filePath:str):
    with open(filePath) as file:
        dataString = file.read()
        dataString = dataString.replace("\n"," ")
        dataString = dataString.replace("name ","name\n")
        dataString = dataString.replace(" ",",")
        return dataString

In [85]:
# read the data from the specified datasets into the df 
from io import StringIO
df = pd.DataFrame()
for dataset in datasets:
    dataset_df = pd.read_csv(StringIO(readRawData("./Data/"+ dataset +".data")), header=None, sep=",")
    dataset_df['dataset'] = dataset
    df = pd.concat([df,dataset_df ], ignore_index=True)

In [86]:
df.columns=["id", "ccf", "age", "sex", "painloc", "painexer" , "relrest" , "pncaden" , "cp", "trestbps", "htn", "chol", "smoke", "cigs", "years", "fbs", "dm", "famhist", "restecg", "ekgmo", "ekgday", "ekgyr", "dig", "prop", "nitr", "pro", "diuretic", "proto", "thaldur", "thaltime", "met", "thalach", "thalrest", "tpeakbps", "tpeakbpd", "dummy", "trestbpd", "exang", "xhypo", "oldpeak", "slope", "rldv5", "rldv5e", "ca", "restckm", "exerckm", "restef", "restwm", "exeref", "exerwm", "thal", "thalsev", "thalpul", "earlobe", "cmo", "cday", "cyr", "num", "lmt", "ladprox", "laddist", "diag", "cxmain", "ramus", "om1", "om2", "rcaprox", "rcadist", "lvx1", "lvx2", "lvx3", "lvx4", "lvf", "cathef", "junk", "name", "dataset"]

In [87]:
# the dataset encodes unfilled cells with -9 they are replaced with NaN for better compatibility with pd 
df = df.replace(-9, float('nan'))

In [88]:
df.describe()

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,htn,chol,smoke,cigs,years,fbs,dm,famhist,restecg,ekgmo,ekgday,ekgyr,dig,prop,nitr,pro,diuretic,proto,thaldur,thaltime,met,thalach,thalrest,tpeakbps,tpeakbpd,dummy,trestbpd,exang,xhypo,oldpeak,slope,rldv5,rldv5e,ca,restckm,exerckm,restef,restwm,exeref,exerwm,thal,thalsev,thalpul,earlobe,cmo,cday,cyr,num,lmt,ladprox,laddist,diag,cxmain,ramus,om1,om2,rcaprox,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk
count,899.000000,899.0,899.000000,899.000000,617.000000,617.000000,613.000000,0.0,899.000000,840.000000,865.000000,869.000000,230.000000,479.000000,467.000000,809.000000,95.000000,477.000000,897.000000,846.000000,845.000000,846.000000,831.000000,833.000000,834.000000,836.000000,817.000000,787.000000,843.000000,446.000000,794.000000,844.000000,843.000000,836.000000,836.000000,840.000000,840.000000,844.000000,841.000000,837.000000,591.000000,474.000000,757.000000,291.000000,0.0,1.0,28.000000,30.000000,2.000000,5.000000,422.000000,130.000000,44.000000,1.0,888.000000,890.000000,890.000000,899.000000,624.000000,663.000000,653.000000,341.000000,664.000000,332.000000,628.000000,327.000000,654.000000,629.000000,880.000000,880.000000,880.000000,880.000000,883.000000,311.000000,119.000000
mean,957.235818,0.0,53.480534,0.790879,0.920583,0.593193,0.672104,NaN,3.253615,132.101190,0.476301,198.759494,0.517391,19.118998,18.796574,0.166873,0.957895,0.563941,0.603122,5.973995,15.493491,84.056738,0.034898,0.283313,0.266187,0.172249,0.112607,37.081321,8.655872,5.690359,16.483123,137.298578,75.487544,171.641148,87.293062,132.101190,83.523810,0.390995,0.026159,0.870490,1.766497,14.398734,54.914135,0.697595,NaN,0.0,0.531071,1.033333,0.550000,0.200000,5.018957,1.284615,0.295455,0.0,6.122748,15.988764,83.839326,1.129032,1.323718,1.327300,1.248086,1.202346,1.296687,1.114458,1.176752,1.067278,1.342508,1.171701,1.020455,1.032955,1.132955,1.611364,1.178935,27.623119,5.869748
std,1204.015482,0.0,9.435894,0.406908,0.270607,0.491637,0.469830,NaN,0.928499,19.151127,0.499727,111.834415,0.500787,18.296273,16.359145,0.373093,0.201895,0.496415,0.803669,3.486479,8.761939,1.640204,0.183631,0.870965,0.442228,0.377823,0.316306,50.144559,3.746617,3.994673,30.772801,25.965959,14.727961,25.734488,14.734586,19.151127,10.252563,0.488263,0.159704,1.080548,0.621339,5.702942,60.309425,1.052728,NaN,NaN,0.146195,1.066200,0.070711,0.447214,1.949388,0.958314,0.461522,NaN,3.474114,8.860872,4.407533,1.259720,6.447542,0.469582,0.432233,0.402339,0.457142,0.318847,0.381762,0.250887,0.474912,0.377421,0.277384,0.415902,0.703837,1.722199,0.512572,31.675295,1.650914
min,1.000000,0.0,28.000000,0.000000,0.000000,0.000000,0.000000,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,81.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,2.000000,60.000000,37.000000,84.000000,11.000000,0.000000,0.000000,0.000000,0.000000,-2.600000,0.000000,2.000000,2.000000,0.000000,NaN,0.0,0.220000,0.000000,0.500000,0.000000,1.000000,0.000000,0.000000,0.0,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.220000,3.300000
25%,116.000000,0.0,47.000000,1.000000,1.000000,0.000000,0.000000,NaN,3.000000,120.000000,0.000000,175.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,3.000000,8.000000,83.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,6.000000,3.000000,5.000000,120.000000,65.000000,155.000000,80.000000,120.000000,80.000000,0.000000,0.000000,0.000000,1.000000,10.000000,12.000000,0.000000,NaN,0.0,0.407500,0.000000,0.525000,0.000000,3.000000,0.000000,0.000000,0.0,3.000000,8.000000,83.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.685000,4.800000
50%,266.000000,0.0,54.000000,1.000000,1.000000,1.000000,1.

In [89]:
df.head()

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,htn,chol,smoke,cigs,years,fbs,dm,famhist,restecg,ekgmo,ekgday,ekgyr,dig,prop,nitr,pro,diuretic,proto,thaldur,thaltime,met,thalach,thalrest,tpeakbps,tpeakbpd,dummy,trestbpd,exang,xhypo,oldpeak,slope,rldv5,rldv5e,ca,restckm,exerckm,restef,restwm,exeref,exerwm,thal,thalsev,thalpul,earlobe,cmo,cday,cyr,num,lmt,ladprox,laddist,diag,cxmain,ramus,om1,om2,rcaprox,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk,name,dataset
0,1254.0,0.0,40.0,1.0,1.0,0.0,0.0,NaN,2.0,140.0,0.0,289.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,12.0,16.0,84.0,0.0,0.0,0.0,0.0,0.0,150.0,18.0,NaN,7.0,172.0,86.0,200.0,110.0,140.0,86.0,0.0,0.0,0.0,NaN,26.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,20.0,84.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
1,1255.0,0.0,49.0,0.0,1.0,0.0,0.0,NaN,3.0,160.0,1.0,180.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,11.0,16.0,84.0,0.0,0.0,0.0,0.0,0.0,NaN,10.0,9.0,7.0,156.0,100.0,220.0,106.0,160.0,90.0,0.0,0.0,1.0,2.0,14.0,13.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,20.0,84.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
2,1256.0,0.0,37.0,1.0,1.0,0.0,0.0,NaN,2.0,130.0,0.0,283.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,11.0,21.0,84.0,0.0,0.0,0.0,0.0,0.0,100.0,10.0,NaN,5.0,98.0,58.0,180.0,100.0,130.0,80.0,0.0,0.0,0.0,NaN,17.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,26.0,84.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
3,1257.0,0.0,48.0,0.0,1.0,1.0,1.0,NaN,4.0,138.0,0.0,214.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,9.0,21.0,84.0,0.0,0.0,0.0,0.0,0.0,50.0,5.0,4.0,4.0,108.0,54.0,210.0,106.0,138.0,86.0,1.0,0.0,1.5,2.0,19.0,22.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,30.0,84.0,3.0,NaN,2.0,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian
4,1258.0,0.0,54.0,1.0,1.0,0.0,1.0,NaN,3.0,150.0,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,7.0,25.0,84.0,0.0,0.0,1.0,1.0,0.0,25.0,2.0,NaN,3.0,122.0,74.0,130.0,100.0,150.0,90.0,0.0,1.0,0.0,NaN,13.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.0,30.0,84.0,0.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,1.0,1.0,1.0,NaN,NaN,name,hungarian


In [90]:
if generate_pandas_profiling_reports:
    profile = ProfileReport(df, title='Pandas Profiling Report for all features')
    profile.to_file("Pandas Profiling Report for all features.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/home/finn/miniconda3/envs/DataMiningProject2/lib/python3.9/site-packages/pandas_profiling/model/correlations.py:55: UserWarning: There was an attempt to calculate the cramers correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"cramers": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/pandas-profiling/issues
(include the error message: 'No data; `observed` has size 0.')
  warnings.warn(
/home/finn/miniconda3/envs/DataMiningProject2/lib/python3.9/site-packages/missingno/missingno.py:254: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax1.set_xticklabels(
/home/finn/miniconda3/envs/DataMiningProject2/lib/python3.9/site-packages/pandas_profiling/model/missing.py:89: UserWarning: There was an attempt to generate the Count missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.p

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

The columns smoke and years both describe whether a respondent smokes or not. Smoke does this by being binary coded, while years describes the number of years a person has smoked. Due to the high number of missing values, the columns are useless on their own. However, it is possible to enrich the smoke column with the years column. 

In [91]:
print(f"Number of NaNs in smoke: {df['smoke'].isna().sum()}")
df.loc[(df['smoke'].isna()) & (df['years'] == 0),'smoke'] = 0
df.loc[(df['smoke'].isna()) & (df['years'] > 0),'smoke'] = 1
df.drop(columns=['smoke'])
print(f"Number of NaNs in smoke after combination with years: {df['smoke'].isna().sum()}")
df.loc[(df['smoke'].isna()) & (df['cigs'] == 0),'smoke'] = 0
df.loc[(df['smoke'].isna()) & (df['cigs'] > 0),'smoke'] = 1
print(f"Number of NaNs in smoke after combination with years and cigs: {df['smoke'].isna().sum()}")

Number of NaNs in smoke: 671
Number of NaNs in smoke after combination with years: 391
Number of NaNs in smoke after combination with years and cigs: 389


Finding: reduces the number of missing values in smoke by 280 entries. 

# Explore how many NaNs are within one coloumn for each attribute.

In [92]:
(df.loc[ : , df.columns != 'dataset'].isna()).join(df['dataset']).groupby("dataset").sum()

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,htn,chol,smoke,cigs,years,fbs,dm,famhist,restecg,ekgmo,ekgday,ekgyr,dig,prop,nitr,pro,diuretic,proto,thaldur,thaltime,met,thalach,thalrest,tpeakbps,tpeakbpd,dummy,trestbpd,exang,xhypo,oldpeak,slope,rldv5,rldv5e,ca,restckm,exerckm,restef,restwm,exeref,exerwm,thal,thalsev,thalpul,earlobe,cmo,cday,cyr,num,lmt,ladprox,laddist,diag,cxmain,ramus,om1,om2,rcaprox,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk,name
dataset,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cleveland,0,0,0,0,282,282,282,282,0,0,0,0,4,5,5,0,259,0,0,0,0,0,2,2,2,2,2,0,0,69,0,0,0,0,0,0,0,0,0,0,0,282,0,2,282,282,282,282,282,282,2,282,282,282,0,0,0,0,0,0,0,282,0,282,0,282,0,0,0,0,0,0,0,282,282,0
hungarian,1,1,1,1,1,1,1,295,1,2,2,24,283,294,295,9,274,294,2,1,1,1,2,3,2,2,2,10,3,191,3,2,2,2,2,2,2,2,3,1,191,2,1,291,295,295,295,292,295,293,267,268,278,295,1,1,1,1,276,237,247,277,236,286,272,290,245,270,1,1,1,1,1,267,295,1
long-beach-va,1,1,1,1,1,1,5,201,1,57,4,8,2,11,13,8,158,9,1,53,54,53,61,61,60,60,74,54,54,161,54,54,55,60,60,57,57,54,54,57,102,66,66,199,201,200,173,174,199,198,160,170,200,200,3,1,1,1,1,1,1,1,1,1,1,2,2,2,3,3,3,3,4,24,82,1
switzerland,0,0,0,0,0,0,0,123,0,2,30,0,100,112,121,75,115,121,1,1,1,1,5,2,3,1,6,50,1,34,50,1,1,3,3,2,2,1,3,6,17,77,77,118,123,123,123,123,123,123,50,51,97,123,9,9,9,0,0,0,0,0,0,0,0,0,0,0,17,17,17,17,13,17,123,0


In [93]:
(df.loc[ : , df.columns != 'dataset'].eq(0)).join(df['dataset']).groupby("dataset").sum()

,id,ccf,age,sex,painloc,painexer,relrest,pncaden,cp,trestbps,htn,chol,smoke,cigs,years,fbs,dm,famhist,restecg,ekgmo,ekgday,ekgyr,dig,prop,nitr,pro,diuretic,proto,thaldur,thaltime,met,thalach,thalrest,tpeakbps,tpeakbpd,dummy,trestbpd,exang,xhypo,oldpeak,slope,rldv5,rldv5e,ca,restckm,exerckm,restef,restwm,exeref,exerwm,thal,thalsev,thalpul,earlobe,cmo,cday,cyr,num,lmt,ladprox,laddist,diag,cxmain,ramus,om1,om2,rcaprox,rcadist,lvx1,lvx2,lvx3,lvx4,lvf,cathef,junk,name
dataset,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
cleveland,0,282,0,91,0,0,0,0,0,0,108,0,115,115,115,240,0,107,138,0,0,0,271,186,211,252,248,0,0,43,0,0,0,0,0,0,0,190,276,91,0,0,0,166,0,0,0,0,0,0,0,0,0,0,0,0,0,157,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
hungarian,0,294,0,81,23,164,141,0,0,0,195,0,10,0,0,266,0,1,235,0,0,0,293,274,265,269,290,0,0,0,0,0,0,0,0,0,0,204,291,189,0,0,0,3,0,0,0,3,0,2,0,6,7,0,0,0,0,188,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
long-beach-va,0,200,0,6,15,65,33,0,0,1,90,49,96,38,38,125,4,100,80,0,0,0,122,88,61,105,94,0,0,2,0,0,0,0,0,1,1,52,139,40,1,0,0,2,0,1,0,10,0,2,0,9,1,1,0,0,0,51,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
switzerland,0,123,0,10,11,22,27,0,0,0,60,123,5,0,0,43,0,0,85,0,0,0,116,70,75,66,93,0,0,25,0,0,0,0,0,0,0,68,113,42,0,0,0,0,0,0,0,0,0,0,0,21,23,0,0,0,0,8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [94]:
columns_to_drop = ["id",
"painloc",
"painexer",
"relrest",
"ccf",
"pncaden",
"years",
"cigs",
"dm",
"famhist",
"ekgmo",
"ekgday",
"ekgyr",
"dig",
"prop",
"nitr",
"diuretic",
"proto",
"thaldur",
"thaltime",
"dummy",
"slope",
"rldv5",
"ca",
"restckm",
"exerckm",
"restef",
"restwm",
"exeref",
"exerwm",
"thal",
"thalsev",
"thalpul",
"earlobe",
"cmo",
"cday",
"cyr",
"lmt",
"ladprox",
"laddist",
"diag",
"cxmain",
"ramus",
"om1",
"om2",
"rcaprox",
"rcadist",
"lvx1",
"lvx2",
"lvx3",
"lvx4",
"lvf",
"cathef",
"junk",
"name"]
df.drop(columns_to_drop, inplace=True, axis=1)
len(columns_to_drop)

55

In [95]:
if generate_pandas_profiling_reports:
    profile = ProfileReport(df, title='Pandas Profiling Report for selected features')
    profile.to_file("Pandas Profiling Report for selected features.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [96]:
df.drop(["smoke","met", "rldv5e"], inplace=True, axis=1)
len(columns_to_drop)

55

In [97]:
df.isna().sum()

age          2
sex          2
cp           2
trestbps    61
htn         36
chol        32
fbs         92
restecg      4
pro         65
thalach     57
thalrest    58
tpeakbps    65
tpeakbpd    65
trestbpd    61
exang       57
xhypo       60
oldpeak     64
num          2
dataset      0
dtype: int64

In [98]:
print(f"Shape before drop of NaN containing rows: {df.shape}")
df.dropna(inplace=True, axis=0, how='any')
print(f"Shape after drop of NaN containing rows: {df.shape}")

Shape before drop of NaN containing rows: (901, 19)
Shape after drop of NaN containing rows: (706, 19)


In [99]:
df.describe()

,age,sex,cp,trestbps,htn,chol,fbs,restecg,pro,thalach,thalrest,tpeakbps,tpeakbpd,trestbpd,exang,xhypo,oldpeak,num
count,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000
mean,53.025496,0.764873,3.225212,132.500000,0.491501,220.641643,0.148725,0.634561,0.148725,138.771955,75.576487,172.182720,86.910765,83.881020,0.398017,0.021246,0.877054,1.067989
std,9.461347,0.424379,0.938681,18.624185,0.500282,94.375922,0.356070,0.840227,0.356070,25.913886,14.680830,24.997345,14.887063,10.208049,0.489836,0.144307,1.072062,1.268683
min,28.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.000000,37.000000,84.000000,11.000000,0.000000,0.000000,0.000000,-1.000000,0.000000
25%,46.000000,1.000000,2.250000,120.000000,0.000000,198.000000,0.000000,0.000000,0.000000,120.000000,65.000000,158.000000,80.000000,80.000000,0.000000,0.000000,0.000000,0.000000
50%,54.000000,1.000000,4.000000,130.000000,0.000000,233.000000,0.000000,0.000000,0.000000,140.000000,74.000000,170.000000,86.000000,80.000000,0.000000,0.000000,0.500000,1.000000
75%,60.000000,1.000000,4.000000,140.000000,1.000000,272.000000,0.000000,1.000000,0.000000,159.750000,84.000000,190.000000,100.000000,90.000000,1.000000,0.000000,1.500000,2.000000
max,77.000000,1.000000,4.000000,200.000000,1.000000,603.000000,1.000000,2.000000,1.000000,202.000000,134.000000,240.000000,134.000000,120.000000,1.000000,1.000000,6.200000,4.000000


In [100]:
df

,age,sex,cp,trestbps,htn,chol,fbs,restecg,pro,thalach,thalrest,tpeakbps,tpeakbpd,trestbpd,exang,xhypo,oldpeak,num,dataset
0,40.0,1.0,2.0,140.0,0.0,289.0,0.0,0.0,0.0,172.0,86.0,200.0,110.0,86.0,0.0,0.0,0.0,0.0,hungarian
1,49.0,0.0,3.0,160.0,1.0,180.0,0.0,0.0,0.0,156.0,100.0,220.0,106.0,90.0,0.0,0.0,1.0,1.0,hungarian
2,37.0,1.0,2.0,130.0,0.0,283.0,0.0,1.0,0.0,98.0,58.0,180.0,100.0,80.0,0.0,0.0,0.0,0.0,hungarian
3,48.0,0.0,4.0,138.0,0.0,214.0,0.0,0.0,0.0,108.0,54.0,210.0,106.0,86.0,1.0,0.0,1.5,3.0,hungarian
5,39.0,1.0,3.0,120.0,0.0,339.0,0.0,0.0,0.0,170.0,86.0,198.0,100.0,80.0,0.0,0.0,0.0,0.0,hungarian
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
893,62.0,1.0,4.0,158.0,1.0,170.0,0.0,1.0,0.0,138.0,86.0,202.0,98.0,90.0,1.0,0.0,0.0,1.0,long-beach-va
894,46.0,1.0,4.0,134.0,1.0,310.0,0.0,0.0,0.0,126.0,88.0,174.0,114.0,90.0,0.0,0.0,0.0,2.0,long-beach-va
895,54.0,0.0,4.0,127.0,0.0,333.0,1.0,1.0,0.0,154.0,83.0,158.0,84.0,78.0,0.0,0.0,0.0,1.0,long-beach-va
897,55.0,1.0,4.0,122.0,1.0,223.0,1.0,1.0,0.0,100.0,74.0,210.0,100.0,70.0,0.0,0.0,0.0,2.0,long-beach-va


In [101]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
df[df['num'] >= 1] = 1
df['num'] = labelEncoder.fit_transform(df['num'])

In [102]:
# Compute the correlation matrix
corr = df.corr()

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, cmap="coolwarm", vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

<AxesSubplot:>

In [103]:
X = df.loc[:,(df.columns!= 'num') & (df.columns != 'dataset')]

y = df['num']

In [104]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from  sklearn.naive_bayes import *
estimators_and_hyperparameters=[
    (CatBoostClassifier(random_state=42, thread_count=-1, silent= True),{}),
    (XGBClassifier(random_state=42, n_jobs=-1),{}),
    (SVC(kernel='linear',random_state=42),{}),
    (SVC(kernel='poly',random_state=42),{}),
    (SVC(kernel='rbf',random_state=42),{}),
    (SVC(kernel='sigmoid',random_state=42),{}),
    #(SVC(kernel='precomputed',random_state=42),{}),
    # (BernoulliNB(),{}),
    #(CategoricalNB(),{}),
    # (ComplementNB(),{}),
    # (GaussianNB(),{}),
    # (MultinomialNB(),{}),
    (DecisionTreeClassifier(random_state=42),{}),
    (KNeighborsClassifier(n_jobs=-1),{}),
    (RandomForestClassifier(random_state=42, n_jobs=-1), {}),
    (SGDClassifier(),{})
]

In [105]:
from sklearn.preprocessing import *
scalers = [
    MaxAbsScaler(),
    MinMaxScaler(),
    Normalizer(),
    PowerTransformer(),
    QuantileTransformer(output_distribution='uniform'),
    QuantileTransformer(output_distribution='normal'),
    RobustScaler(),
    StandardScaler(),
]

In [106]:
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import cross_val_score
# from statistics import mean
# for scaler in scalers:
#     print(f'Current Sclaer: {scaler.__class__.__name__}')
#     for estimator in estimators_and_hyperparameters:
#         skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
#         try:
#             X_trans = scaler.fit_transform(X)
#             scores = cross_val_score(estimator[0], X_trans, y, scoring='f1',cv=skf, n_jobs=-1)
#             print(f'F1 score for {estimator[0].__class__.__name__}: {mean(scores)}')
#         except Exception as e:
#             print(e)
#             print(f'Skipping the combination of {scaler.__class__.__name__} and {estimator.__class__.__name__}')
#     print('-----------------------------------------------------------------')

# Tests with the preprocessed data by the UCI

In [107]:
import pandas as pd 
datasets = ["hungarian", "cleveland", "switzerland", "va"]
df_processed = pd.DataFrame()
for dataset in datasets:
    dataset_df = pd.read_csv("./Data/processed."+ dataset +".data", header=None, sep=",")
    dataset_df['dataset'] = dataset
    df_processed = pd.concat([df_processed,dataset_df ], ignore_index=True)
df_processed.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num', 'dataset']
df_processed = df_processed.replace('?', float('nan'))


In [108]:
df_processed[['trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']] = df_processed[['trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal']].apply(pd.to_numeric)

In [109]:
(df_processed.loc[ : , df_processed.columns != 'dataset'].isna()).join(df_processed['dataset']).groupby("dataset").sum()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
dataset,,,,,,,,,,,,,,
cleveland,0,0,0,0,0,0,0,0,0,0,0,4,2,0
hungarian,0,0,0,1,23,8,1,1,1,0,190,291,266,0
switzerland,0,0,0,2,0,75,1,1,1,6,17,118,52,0
va,0,0,0,56,7,7,0,53,53,56,102,198,166,0


In [110]:
df_processed.drop(["slope", "ca","thal"], inplace=True, axis=1)

In [111]:
print(f"Shape before drop of NaN containing rows: {df_processed.shape}")
df_processed.dropna(inplace=True, axis=0, how='any')
print(f"Shape after drop of NaN containing rows: {df_processed.shape}")

Shape before drop of NaN containing rows: (920, 12)
Shape after drop of NaN containing rows: (740, 12)


In [112]:
df.shape

(706, 19)

In [113]:
df_processed.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,num,dataset
0,28.0,1.0,2.0,130.0,132.0,0.0,2.0,185.0,0.0,0.0,0,hungarian
1,29.0,1.0,2.0,120.0,243.0,0.0,0.0,160.0,0.0,0.0,0,hungarian
3,30.0,0.0,1.0,170.0,237.0,0.0,1.0,170.0,0.0,0.0,0,hungarian
4,31.0,0.0,2.0,100.0,219.0,0.0,1.0,150.0,0.0,0.0,0,hungarian
5,32.0,0.0,2.0,105.0,198.0,0.0,0.0,165.0,0.0,0.0,0,hungarian


In [114]:
df_processed.describe()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,num
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,53.097297,0.764865,3.227027,132.754054,220.136486,0.150000,0.635135,138.744595,0.400000,0.894324,0.924324
std,9.408127,0.424370,0.939193,18.581250,93.614555,0.357313,0.840039,25.846082,0.490229,1.087160,1.128882
min,28.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,60.000000,0.000000,-1.000000,0.000000
25%,46.000000,1.000000,2.000000,120.000000,197.000000,0.000000,0.000000,120.000000,0.000000,0.000000,0.000000
50%,54.000000,1.000000,4.000000,130.000000,231.000000,0.000000,0.000000,140.000000,0.000000,0.500000,1.000000
75%,60.000000,1.000000,4.000000,140.000000,271.000000,0.000000,1.000000,159.250000,1.000000,1.500000,1.000000
max,77.000000,1.000000,4.000000,200.000000,603.000000,1.000000,2.000000,202.000000,1.000000,6.200000,4.000000


In [115]:
if generate_pandas_profiling_reports:
    profile = ProfileReport(df, title='Pandas Profiling Report for the features processed by the UCI')
    profile.to_file("Pandas Profiling Report for the features processed by the UCI.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [116]:
# Tests with the preprocessed data by the UCI includeing the reprocessed data

In [117]:
import pandas as pd 
datasets = ["hungarian", "cleveland", "switzerland", "va"]
df_processed = pd.DataFrame()
for dataset in datasets:
    if dataset != "hungarian":
        dataset_df = pd.read_csv("./Data/processed."+ dataset +".data", header=None, sep=",")
        dataset_df['dataset'] = dataset
        df_processed = pd.concat([df_processed,dataset_df ], ignore_index=True)
with open("Data/reprocessed.hungarian.data") as file:
    dataString = file.read()
    dataString = dataString.replace(" ",",")
    dataset_df = pd.read_csv(StringIO(dataString), header=None, sep=",")
    dataset_df['dataset'] = dataset
df_processed = pd.concat([df_processed,dataset_df ], ignore_index=True)
df_processed.columns = ['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang','oldpeak','slope','ca','thal','num', 'dataset']
df_processed = df_processed.replace('?', float('nan'))

In [118]:
df_processed

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num,dataset
0,63.0,1.0,1.0,145.0,233.0,1.0,2.0,150.0,0.0,2.3,3.0,0.0,6.0,0.0,cleveland
1,67.0,1.0,4.0,160.0,286.0,0.0,2.0,108.0,1.0,1.5,2.0,3.0,3.0,2.0,cleveland
2,67.0,1.0,4.0,120.0,229.0,0.0,2.0,129.0,1.0,2.6,2.0,2.0,7.0,1.0,cleveland
3,37.0,1.0,3.0,130.0,250.0,0.0,0.0,187.0,0.0,3.5,3.0,0.0,3.0,0.0,cleveland
4,41.0,0.0,2.0,130.0,204.0,0.0,2.0,172.0,0.0,1.4,1.0,0.0,3.0,0.0,cleveland
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
916,36.0,1.0,2.0,120.0,166.0,0.0,0.0,180.0,0.0,0.0,-9.0,-9.0,-9.0,0.0,va
917,48.0,1.0,3.0,110.0,211.0,0.0,0.0,138.0,0.0,0.0,-9.0,-9.0,6.0,0.0,va
918,47.0,0.0,2.0,140.0,257.0,0.0,0.0,135.0,0.0,1.0,1.0,-9.0,-9.0,0.0,va
919,53.0,1.0,4.0,130.0,182.0,0.0,0.0,148.0,0.0,0.0,-9.0,-9.0,-9.0,0.0,va


In [119]:
if generate_pandas_profiling_reports:
    profile = ProfileReport(df, title='Pandas Profiling Report for the features processed by the UCI + reprocessed hungarian')
    profile.to_file("Pandas Profiling Report for the features processed by the UCI + reprocessed hungarian.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]